In [15]:
!curl localhost:9200

{
  "name" : "4466ceb0d02d",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "UpermVuuQMCqS86Tp3z_sQ",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   541  100   541    0     0   3310      0 --:--:-- --:--:-- --:--:--  3319


In [10]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [2]:
from elasticsearch8 import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "rag-course-questions-homework"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag-course-questions-homework'})

In [13]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [14]:
query = "How do execute a command on a Kubernetes pod?"

In [15]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [16]:
result_scores = []
for hit in response['hits']['hits']:
    result_scores.append(hit['_score'])

In [17]:
result_scores

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

In [18]:
query = "How do copy a file to a Docker container?"

In [19]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [22]:
response

ObjectApiResponse({'took': 105, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 340, 'relation': 'eq'}, 'max_score': 73.38676, 'hits': [{'_index': 'rag-course-questions-homework', '_id': 'OeyMUJcBXi85EXaVhWkl', '_score': 73.38676, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'rag-course-questions-homework', '_id': 'WOyMUJcBXi85EXaVjGmR', '_score': 66.688705, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command.

In [30]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source']['question'])

In [31]:
for result_doc in result_docs:
    print(result_doc)

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [26]:
def search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [40]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context_template = """
Q: {question}
A: {text}
""".strip()
    context = ""
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']).strip() + "\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [41]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [42]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

In [39]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: How do copy a file to a Docker container?

        CONTEXT:
        Q: How do I debug a docker container?
        A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
        A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local

In [43]:
len(prompt)

1446

In [44]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [45]:
len(encoding.encode(prompt))

320

In [46]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

In [47]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [48]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [50]:
print(rag(query))

You can copy a file from your local machine into a Docker container using the `docker cp` command. The basic syntax is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```
